In [ ]:
import importlib
from IPython.core.magic import register_cell_magic
from IPython import get_ipython
# Conditional skipping of https://kioku-space.com/en/jupyter-skip-execution/
@register_cell_magic
def skip_if(line, cell):
    if eval(line):
        return
    get_ipython().run_cell(cell)

get_ipython().register_magic_function(skip_if, 'cell')

In [ ]:
import mercury as mr
import nbformat

In [ ]:
app = mr.App(
    title="CS 365: Transit Regina Data Wrangling", 
    description="Cleaning and transforming Transit Regina data for analysis", 
    show_code=False
)

  # CS 365 Final Project: Transit Regina Data Wrangling

 ## Janice Cotcher
 ## December 5, 2025

 **Data Source:** City of Regina Open Data Portal

 **Dataset:** Transit Stops and Routes (November 20, 2025)

 ## Dataset Source & License

  **Source:** [City of Regina Open Data Portal](https://open.regina.ca)
  - Bus Stop Locations (yqrStops.json)
  - Transit Routes (yqrRoutes.json)
  - General Transit Feed Specification(routes.txt, stops.txt, trips.txt, stop_times.txt)

  **License:** Open Government License - Regina
 - Allows educational and commercial use
 - No Personally Identifiable Information - only public infrastructure data

  **Why this matters:** Understanding transit accessibility and route coverage for urban planning

In [ ]:
from IPython.display import Image
Image('open_regina_license.png')

 ## Why I used a Jupyter Notebook

 - Technical Skills
 - Dataset size[1]
 - Workflow





 [1] CS 365 - Data Cleaning: Concepts & Algorithms, Lecture 6: September 15, 2025

 ## Imported Python Libraries

 ```python
 import numpy as np # scientific computing for large, multi-dimensional arrays
 from matplotlib import pyplot as plt # creates static, animated, and interactive visualizations
 import pandas as pd #creates 2D, size-mutable, heterogeneous tables called data frames
 import json # read and write json files
 import plotly.graph_objects as go # interactive graphics like maps
 from pyproj import Transformer # cartography and coordinate transformations
 ```

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import json
import plotly.graph_objects as go
from pyproj import Transformer

In [ ]:
mr.Markdown("""## Raw Data Snapshot: Bus Stops""")
Image('stops_raw_data.png')

In [ ]:
mr.Markdown("""
```python
# Load JSON data
try:
    with open('raw_data/yqrStops.json', 'r') as f:
        stop_data = json.load(f)
except json.decoder.JSONDecodeError as e:
    print("Invalid JSON", e)

# Normalize nested JSON structure
df_stops = pd.json_normalize(stop_data['features'])
```
""")

In [ ]:
# Load stop data
try:
    with open('raw_data/yqrStops.json', 'r') as f:
        stop_data = json.load(f)
except json.decoder.JSONDecodeError as e:
    print("Invalid JSON", e)

df_stops = pd.json_normalize(stop_data['features'])

mr.Markdown(f"**Loaded bus stops:**{len(df_stops)}")
df_stops.head(10)

 # Data Profiling: Quality Assessment - Stops

 - **Data types** - Are coordinates stored correctly?
 - **Missing values** - Which columns have gaps?
 - **Duplicates** - Any duplicate stop IDs?
 - **Outliers** - Any stops in unexpected locations?
 - **Cardinalities** - How many unique stops?

 ### BUS STOP DATA TYPES

In [ ]:
print(df_stops.dtypes[df_stops.columns[:8]])

 ## Missing Values

In [ ]:
missing = df_stops.isnull().sum()
print(f"Number of Missing Values: {len(missing[missing > 0])}")
print("Missing Values")
print(missing)

 ## Duplicates

In [ ]:
duplicates = df_stops.duplicated(subset=['attributes.STOP_ID']).sum()
print(f"Number of Duplicate Stops - checked by stop ID: {duplicates}")

 ### Sample Latitude Values

In [ ]:
df_stops['attributes.LAT'].head(5).tolist()

 # Cleaning: Bus Stop Text Standardization & Type Conversions

In [ ]:
mr.Markdown("""
## Problems identified
- Inconsistent text formatting (mixed case, whitespace)
- Coordinates stored as strings instead of numeric latitude and longitude
- Missing values in street names
""")

In [ ]:
mr.Markdown("""
**Sample Solutions:**
```python
# Text standardization
df_stops['attributes.ONSTREET'] = df_stops['attributes.ONSTREET'].str.strip().str.upper()
df_stops['attributes.ATSTREET'] = df_stops['attributes.ATSTREET'].str.strip().str.upper()
...
# Missing value imputation
df_stops = df_stops.fillna({'attributes.ATSTREET': "DOROTHY ST (SB)"})

# Data correction
df_stops['attributes.ONSTREET'] = df_stops['attributes.ONSTREET'].str.replace(
    "1060 DOROTHY ST (SB)", "DOROTHY ST", regex=False
)
```
""")

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_stops):
    # Remove leading and trailing whitespace in column: 'attributes.ONSTREET'
    df_stops['attributes.ONSTREET'] = df_stops['attributes.ONSTREET'].str.strip()
    # Convert text to uppercase in column: 'attributes.ONSTREET'
    df_stops['attributes.ONSTREET'] = df_stops['attributes.ONSTREET'].str.upper()
    # Remove leading and trailing whitespace in column: 'attributes.ATSTREET'
    df_stops['attributes.ATSTREET'] = df_stops['attributes.ATSTREET'].str.strip()
    # Convert text to uppercase in column: 'attributes.ATSTREET'
    df_stops['attributes.ATSTREET'] = df_stops['attributes.ATSTREET'].str.upper()
    # Remove leading and trailing whitespace in column: 'attributes.LON'
    df_stops['attributes.LON'] = df_stops['attributes.LON'].str.strip()
    # Remove leading and trailing whitespace in column: 'attributes.LAT'
    df_stops['attributes.LAT'] = df_stops['attributes.LAT'].str.strip()
    # Remove leading and trailing whitespace in column: 'stop_id'
    df_stops['attributes.STOP_ID'] = df_stops['attributes.STOP_ID'].str.strip()
    # Remove leading and trailing whitespace in column: 'attributes.STOP_NAME'
    df_stops['attributes.STOP_NAME'] = df_stops['attributes.STOP_NAME'].str.strip()
    # Convert text to uppercase in column: 'attributes.STOP_NAME'
    df_stops['attributes.STOP_NAME'] = df_stops['attributes.STOP_NAME'].str.upper()
    # Remove leading and trailing whitespace in column: 'attributes.GLOBALID'
    df_stops['attributes.GLOBALID'] = df_stops['attributes.GLOBALID'].str.strip()
    # Replace missing values with "DOROTHY ST (SB)" in column: 'attributes.ATSTREET'
    df_stops = df_stops.fillna({'attributes.ATSTREET':"DOROTHY ST (SB)"})
    # Replace all instances of "1060 DOROTHY ST (SB)" with "DOROTHY ST" in column: 'attributes.ONSTREET'
    df_stops['attributes.ONSTREET'] = df_stops['attributes.ONSTREET'].str.replace("1060 DOROTHY ST (SB)", "DOROTHY ST", case=False, regex=False)
    # Rename column 'attributes.ONSTREET' to 'on_street'
    df_stops = df_stops.rename(columns={'attributes.ONSTREET': 'on_street'})
    # Rename column 'attributes.ATSTREET' to 'at_street'
    df_stops = df_stops.rename(columns={'attributes.ATSTREET': 'at_street'})
    # Rename column 'attributes.LON' to 'lon'
    df_stops = df_stops.rename(columns={'attributes.LON': 'lon'})
    # Rename column 'attributes.LAT' to 'lat'
    df_stops = df_stops.rename(columns={'attributes.LAT': 'lat'})
    # Rename column 'stop_id' to 'stop_id'
    df_stops = df_stops.rename(columns={'attributes.STOP_ID': 'stop_id'})
    # Rename column 'attributes.STOP_NAME' to 'stop_name'
    df_stops = df_stops.rename(columns={'attributes.STOP_NAME': 'stop_name'})
    # Rename column 'attributes.GLOBALID' to 'global_id'
    df_stops = df_stops.rename(columns={'attributes.GLOBALID': 'global_id'})
    # Rename column 'attributes.OBJECTID' to 'object_id'
    df_stops = df_stops.rename(columns={'attributes.OBJECTID': 'object_id'})
    return df_stops

clean_stops= clean_data(df_stops.copy())
clean_stops.head()

mr.Markdown(f"**Cleaned {len(clean_stops)} bus stops**")
clean_stops.head()

 ## Raw Data Snapshot: Bus Routes

In [ ]:
Image('routes_raw_data_1.png')

In [ ]:
Image('routes_raw_data_2.png')

In [ ]:
mr.Markdown("""## Over 90K lines, ~4.9MB""")
Image('routes_raw_data_3.png')

  ## Loading Routes Data

In [ ]:
mr.Markdown("""
```python
# Load routes data
with open('raw_data/yqrRoutes.json', 'r') as f:
    routes_data = json.load(f)

df_routes = pd.json_normalize(routes_data['features'])
```
""")

In [ ]:
# Load routes
try:
    with open('raw_data/yqrRoutes.json', 'r') as f:
        routes_data = json.load(f)
except json.decoder.JSONDecodeError as e:
    print("Invalid JSON", e)

df_routes = pd.json_normalize(routes_data['features'])

In [ ]:
df_routes

 # Data Profile: Quality Assessment - Routes
 - **Data types** - Are coordinates stored correctly?
 - **Missing values** - Which columns have gaps?
 - **Duplicates** - Any duplicate route IDs?
 - **Outliers** - Any routes in unexpected locations?
 - **Cardinalities** - How many unique routes?


In [ ]:
mr.Markdown("""## Route Data Types""")

In [ ]:
print(df_routes.dtypes[df_routes.columns[:6]])

In [ ]:
mr.Markdown("""## Missing Values""")

In [ ]:
missing = df_routes.isnull().sum()
print(f"Number of Missing Values: {missing[missing > 0] if len(missing[missing > 0]) > 0 else "No missing values"}")
print("Missing Values")
print(missing)

In [ ]:
mr.Markdown("""## DUPLICATES""")

In [ ]:
duplicates = df_routes.duplicated(subset=['attributes.ROUTE_ID']).sum()
print(f"Duplicate route IDs: {duplicates}")

 # Cleaning: Bus Route Text Standardization & Type Conversions

 ## Data Wrangler Demo

 ## Bus Route Cleaning Summary
 - inconsistent text formatting
 - missing colours
 - hex values missing #
 - standardize column names

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_routes):
    # Remove leading and trailing whitespace in columns: 'attributes.ROUTE_NAME', 'attributes.ROUTE_NUM' and 4 other columns
    df_routes['attributes.ROUTE_NAME'] = df_routes['attributes.ROUTE_NAME'].str.strip()
    df_routes['attributes.ROUTE_NUM'] = df_routes['attributes.ROUTE_NUM'].str.strip()
    df_routes['attributes.ROUTE_ID'] = df_routes['attributes.ROUTE_ID'].str.strip()
    df_routes['attributes.SHAPE_ID'] = df_routes['attributes.SHAPE_ID'].str.strip()
    # add a hashtag in front of the route colour hex values 
    df_routes['attributes.ROUTE_COLOR'] = '#' + (df_routes['attributes.ROUTE_COLOR'].str.strip()).astype(str)
    # Convert text to uppercase in column: 'attributes.ROUTE_NAME'
    df_routes['attributes.ROUTE_NAME'] = df_routes['attributes.ROUTE_NAME'].str.upper()
    # Replace missing values with "FFFFFF" in column: 'attributes.ROUTE_TEXT_COLOR'
    df_routes = df_routes.fillna({'attributes.ROUTE_TEXT_COLOR': "FFFFFF"})
    # add a hashtag in front of the route text colour hex values
    df_routes['attributes.ROUTE_TEXT_COLOR'] = "#"+ (df_routes['attributes.ROUTE_TEXT_COLOR'].str.strip()).astype(str)
     # Rename column 'attributes.SHAPE.LEN' to 'shape_length'
    df_routes = df_routes.rename(columns={'attributes.SHAPE.LEN': 'shape_length'})
    # Rename column 'attributes.ROUTE_NAME' to 'route_name'
    df_routes = df_routes.rename(columns={'attributes.ROUTE_NAME': 'route_name'})
    # Rename column 'attributes.ROUTE_NUM' to 'route_num'
    df_routes = df_routes.rename(columns={'attributes.ROUTE_NUM': 'route_num'})
    # Rename column 'attributes.ROUTE_ID' to 'route_id'
    df_routes = df_routes.rename(columns={'attributes.ROUTE_ID': 'route_id'})
    # Rename column 'attributes.SHAPE_ID' to 'shape_id'
    df_routes = df_routes.rename(columns={'attributes.SHAPE_ID': 'shape_id'})
    # Rename column 'attributes.ROUTE_COLOR' to 'route_color'
    df_routes = df_routes.rename(columns={'attributes.ROUTE_COLOR': 'route_color'})
    # Rename column 'attributes.ROUTE_TEXT_COLOR' to 'route_text_color'
    df_routes = df_routes.rename(columns={'attributes.ROUTE_TEXT_COLOR': 'route_text_color'})
    # Rename column 'geometry.paths' to 'geometry_paths'
    df_routes = df_routes.rename(columns={'geometry.paths': 'geometry_paths'})
    # Rename column 'attributes.OBJECTID' to 'object_id'
    df_routes = df_routes.rename(columns={'attributes.OBJECTID': 'object_id'})
    return df_routes

clean_routes = clean_data(df_routes.copy())
clean_routes.head()

mr.Markdown(f"**Cleaned routes:**{len(clean_routes)}")
clean_routes[['route_num', 'route_name', 'route_color']].head()

  ## Transformation Step 1: Loading GTFS Schedule Data

In [ ]:
mr.Markdown("""
**GTFS (General Transit Feed Specification)** provides detailed schedule information:

```python
# Load GTFS files
stops_gtfs = pd.read_csv('raw_data/gtfs_data/stops.txt')
routes_gtfs = pd.read_csv('raw_data/gtfs_data/routes.txt')
trips_gtfs = pd.read_csv('raw_data/gtfs_data/trips.txt')
times_gtfs = pd.read_csv('raw_data/gtfs_data/stop_times.txt')

# Clean and standardize - sample
stops_gtfs['stop_name'] = stops_gtfs['stop_name'].str.upper().str.strip()
routes_gtfs['route_long_name'] = routes_gtfs['route_long_name'].str.upper().str.strip()
```
""")

In [ ]:
# Load GTFS data
stops_gtfs = pd.read_csv('raw_data/gtfs_data/stops.txt')
routes_gtfs = pd.read_csv('raw_data/gtfs_data/routes.txt')
trips_gtfs = pd.read_csv('raw_data/gtfs_data/trips.txt')
times_gtfs = pd.read_csv('raw_data/gtfs_data/stop_times.txt')

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(stops_gtfs):
    # Convert text to uppercase in column: 'stop_name'
    stops_gtfs['stop_name'] = stops_gtfs['stop_name'].str.upper()
    # Remove leading and trailing whitespace in column: 'stop_name'
    stops_gtfs['stop_name'] = stops_gtfs['stop_name'].str.strip()
    return stops_gtfs

stops_gtfs_clean = clean_data(stops_gtfs.copy())
stops_gtfs_clean.head()

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(routes_gtfs):
    # Convert text to uppercase in column: 'route_long_name'
    routes_gtfs['route_long_name'] = routes_gtfs['route_long_name'].str.upper()
    # Remove leading and trailing whitespace in column: 'route_long_name'
    routes_gtfs['route_long_name'] = routes_gtfs['route_long_name'].str.strip()
    return routes_gtfs

routes_gtfs_clean = clean_data(routes_gtfs.copy())
routes_gtfs_clean.head()

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(trips_gtfs):
    # Remove leading and trailing whitespace in columns: 'route_id', 'service_id' and 2 other columns
    trips_gtfs['route_id'] = trips_gtfs['route_id'].str.strip()
    trips_gtfs['service_id'] = trips_gtfs['service_id'].str.strip()
    trips_gtfs['trip_id'] = trips_gtfs['trip_id'].str.strip()
    trips_gtfs['trip_headsign'] = trips_gtfs['trip_headsign'].str.strip()
    # Convert text to uppercase in columns: 'service_id', 'trip_id', 'trip_headsign'
    trips_gtfs['service_id'] = trips_gtfs['service_id'].str.upper()
    trips_gtfs['trip_id'] = trips_gtfs['trip_id'].str.upper()
    trips_gtfs['trip_headsign'] = trips_gtfs['trip_headsign'].str.upper()
    return trips_gtfs

trips_gtfs_clean = clean_data(trips_gtfs.copy())
trips_gtfs_clean.head()

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(times_gtfs):
    # Convert text to uppercase in column: 'trip_id'
    times_gtfs['trip_id'] = times_gtfs['trip_id'].str.upper()
    # Replace '24' in hour position with '00'
    times_gtfs['arrival_time'] = times_gtfs['arrival_time'].str.replace(r'^24', '00', regex=True)
    times_gtfs['departure_time'] = times_gtfs['departure_time'].str.replace(r'^24', '00', regex=True)
    # Remove leading and trailing whitespace in columns: 'trip_id', 'arrival_time', 'departure_time'
    times_gtfs['trip_id'] = times_gtfs['trip_id'].str.strip()
    times_gtfs['arrival_time'] = times_gtfs['arrival_time'].str.strip()
    times_gtfs['departure_time'] = times_gtfs['departure_time'].str.strip()
    return times_gtfs

times_gtfs_clean = clean_data(times_gtfs.copy())
times_gtfs_clean.head()

In [ ]:
mr.Markdown(f"""
**GTFS Data Loaded:**
- {len(stops_gtfs_clean)} stops
- {len(routes_gtfs_clean)} routes
- {len(trips_gtfs_clean)} trips
- {len(times_gtfs_clean)} stop times
""")

  ## Transformation Step 2: Parsing Date/Time Data

In [ ]:
mr.Markdown("""
**Problem:** Time data stored as strings (HH:MM:SS)

**Solution:** Parse to datetime and derive time-based features

```python
# Parse time columns
times_gtfs_clean['departure_datetime'] = pd.to_datetime(
    times_gtfs_clean['departure_time'], 
    format='%H:%M:%S',
    errors='coerce'
)
            
times_gtfs_clean['departure_datetime'] = pd.to_datetime(
    times_gtfs_clean['departure_time'], 
    format='%H:%M:%S',
    errors='coerce'
)

# Derive hour and minute features
times_gtfs_clean['arrival_hour'] = times_gtfs_clean['arrival_datetime'].dt.hour
times_gtfs_clean['arrival_minute'] = times_gtfs_clean['arrival_datetime'].dt.minute
times_gtfs_clean['departure_hour'] = times_gtfs_clean['departure_datetime'].dt.hour
times_gtfs_clean['departure_minute'] = times_gtfs_clean['departure_datetime'].dt.minute
```
""")

In [ ]:
# Parse time columns
times_gtfs_clean['arrival_datetime'] = pd.to_datetime(
    times_gtfs_clean['arrival_time'], 
    format='%H:%M:%S',
    errors='coerce'
)

times_gtfs_clean['departure_datetime'] = pd.to_datetime(
    times_gtfs_clean['departure_time'], 
    format='%H:%M:%S',
    errors='coerce'
)

# Derive hour of day
times_gtfs_clean['arrival_hour'] = times_gtfs_clean['arrival_datetime'].dt.hour
times_gtfs_clean['arrival_minute'] = times_gtfs_clean['arrival_datetime'].dt.minute
times_gtfs_clean['departure_hour'] = times_gtfs_clean['departure_datetime'].dt.hour
times_gtfs_clean['departure_minute'] = times_gtfs_clean['departure_datetime'].dt.minute

mr.Markdown("**Parsed time data and derived hour/minute features**")
times_gtfs_clean[['arrival_time', 'arrival_datetime', 'arrival_hour', 'arrival_minute', 'departure_datetime', 'departure_time', 'departure_hour', 'departure_minute']].head(10)

  ## Transformation Step 3: Merge/Join Operations
 - 1400 stops
 - Open Regina ASP.NET limit of 1000
 - GTFS data contains some fields
 - Imputation for the remaining

In [ ]:
print(len(clean_stops))
print(len(stops_gtfs_clean))
if len(stops_gtfs_clean) > len(clean_stops):
    print(f"There are {len(stops_gtfs_clean)-len(clean_stops)} missing stops")

In [ ]:
# make sure the join keys have the same data type
clean_stops['stop_id'] = clean_stops['stop_id'].astype(str)
stops_gtfs_clean['stop_id'] = stops_gtfs_clean['stop_id'].astype(str)

In [ ]:
# find the missing stops
missing_stops = stops_gtfs_clean[~stops_gtfs_clean['stop_id'].isin(clean_stops['stop_id'])]
len(missing_stops)

In [ ]:
mr.Markdown("""
```python
# Find stops in GTFS but not in geographic JSON data
missing_stops = stops_gtfs_clean[
    ~stops_gtfs_clean['stop_id'].isin(clean_stops['stop_id'])
]

# Merge datasets
merged_stops = pd.concat([clean_stops, missing_stops], ignore_index=True, sort=False)

# Impute missing street names from stop_name
for index, stop in merged_stops.iterrows():
    if pd.isna(stop['on_street']):
        merged_stops.at[index, 'on_street'] = stop['stop_name'].split(' @')[0]
    if pd.isna(stop['at_street']):
        merged_stops.at[index, 'at_street'] = stop['stop_name'].split('@ ')[-1]
```
""")

In [ ]:
# Find missing stops
missing_stops = stops_gtfs_clean[~stops_gtfs_clean['stop_id'].isin(clean_stops['stop_id'])]

mr.Markdown(f"**Found stops in GTFS data:***{len(missing_stops)}")

# Merge
merged_stops = pd.concat([clean_stops, missing_stops], ignore_index=True, sort=False)
print(len(missing_stops), "new stops added")

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(merged_stops):
    for index, stop in merged_stops.iterrows():
        if pd.isna(stop['on_street']):
            merged_stops.at[index, 'on_street'] = (
                str(merged_stops.at[index, 'stop_name']).split(' @')[0]
            )
        if pd.isna(stop['at_street']):
            merged_stops.at[index, 'at_street'] = (
                str(merged_stops.at[index, 'stop_name']).split('@ ')[-1]
            )
    return merged_stops

merged_stops_clean = clean_data(merged_stops.copy())

In [ ]:
mr.Markdown(f"**Total stops after merge:** {len(merged_stops_clean)} ({len(missing_stops)} added)")
merged_stops_clean.head()

  ## Transformation: Feature Derivation - Geographic Regions

 **Divide the city into four quadrants** based on approximate city centre coordinates (Albert St & Victoria Ave Intersection)

In [ ]:
mr.Markdown("""
```python
city_centre_lon = -104.618
city_centre_lat = 50.447
merged_stops_clean['region'] = ''

# Assign quadrants (NE, NW, SE, SW)
for stop in range(len(merged_stops_clean)):
    if float(merged_stops_clean['lat'].iloc[stop]) > city_centre_lat:
        if float(merged_stops_clean['lon'].iloc[stop]) > city_centre_lon:
            merged_stops_clean.at[stop, 'region'] = "NE"
        else:
            merged_stops_clean.at[stop, 'region'] = "NW"
    else:
        if float(merged_stops_clean['lon'].iloc[stop]) > city_centre_lon:
            merged_stops_clean.at[stop, 'region'] = "SE"
        else:
            merged_stops_clean.at[stop, 'region'] = "SW"
```
""")

In [ ]:
# Derive Bus Stops by Region
city_centre_lon = -104.618
city_centre_lat = 50.447
merged_stops_clean['region'] = ''

# Assign quadrants (NE, NW, SE, SW)
for stop in range(len(merged_stops_clean)):
    if float(merged_stops_clean['lat'].iloc[stop]) > city_centre_lat:
        if float(merged_stops_clean['lon'].iloc[stop]) > city_centre_lon:
            merged_stops_clean.at[stop, 'region'] = "NE"
        else:
            merged_stops_clean.at[stop, 'region'] = "NW"
    else:
        if float(merged_stops_clean['lon'].iloc[stop]) > city_centre_lon:
            merged_stops_clean.at[stop, 'region'] = "SE"
        else:
            merged_stops_clean.at[stop, 'region'] = "SW"

In [ ]:
mr.Markdown("**Derived regional classifications for all stops**")

# Show distribution of stops per region
region_counts = merged_stops_clean['region'].value_counts()
mr.Markdown(f"""
**Stop distribution by region:**
- NW: {region_counts.get('NW', 0)} stops
- NE: {region_counts.get('NE', 0)} stops
- SW: {region_counts.get('SW', 0)} stops
- SE: {region_counts.get('SE', 0)} stops
""")

  ## Transformation 5: Feature Derivation - Distance Calculations

  - In ArcGIS/GIS systems, shape.len (shape length) represents the total length of the geometry in metres

In [ ]:
mr.Markdown("""
```python
   # convert shape_length into km
   clean_routes['route_distance_km'] = clean_routes['shape_length'] / 1000         
```
""")
clean_routes['route_distance_km'] = clean_routes['shape_length'] / 1000

In [ ]:
clean_routes[['route_num', 'route_distance_km']]

In [ ]:
mr.Markdown("""
**Calculate distance from city centre** using coordinate geometry:

```python
# Approximate conversion: ~111 km per degree latitude, ~85 km per degree longitude at this latitude
# Euclidean distance
merged_stops_clean['distance_from_centre_km'] = np.sqrt(
    ((merged_stops_clean['lat'].astype(float) - city_centre_lat) * 111)**2 + 
    ((merged_stops_clean['lon'].astype(float) - city_centre_lon) * 85)**2
)
```
""")

In [ ]:
# Approximate conversion: ~111 km per degree latitude, ~85 km per degree longitude at this latitude
# Euclidean distance
merged_stops_clean['distance_from_centre_km'] = np.sqrt(
    ((merged_stops_clean['lat'].astype(float) - city_centre_lat) * 111)**2 + 
    ((merged_stops_clean['lon'].astype(float) - city_centre_lon) * 85)**2
)

mr.Markdown("**Calculated distance from city centre for all stops**")
print("\nDistance statistics (km):")
print(merged_stops_clean['distance_from_centre_km'].describe())

  ## Transformation 6: Aggregation

In [ ]:
mr.Markdown("""
**Aggregate stops by region** to understand service distribution:

```python
region_summary = clean_stops.groupby('region').agg({
    'stop_id': 'count',
    'distance_from_centre_km': ['mean', 'max']
}).reset_index()
```
""")

In [ ]:
region_summary = merged_stops_clean.groupby('region').agg({
    'stop_id': 'count',
    'distance_from_centre_km': ['mean', 'max']
}).reset_index()

In [ ]:
region_summary.columns = ['region', 'num_stops', 'avg_distance_km', 'max_distance_km']

mr.Markdown("**Regional Summary Statistics:**")
region_summary

  ## Transformation 7: Reshape (Pivot)

In [ ]:
mr.Markdown("""
**Pivot analysis:** How many stops does each route serve in each region?

```python
# Join stop times → trips → stops to get route-region relationships
route_stops = (
    times_gtfs_clean
    .merge(trips_gtfs_clean[['trip_id', 'route_id']], on='trip_id')
    .merge(merged_stops_clean[['stop_id', 'region']], left_on='stop_id', right_on='stop_id')
    .groupby(['route_id', 'region'])
    .size()
    .reset_index(name='stop_count')
)

# Pivot to wide format
route_region_pivot = route_stops.pivot(
    index='route_id', 
    columns='region', 
    values='stop_count'
).fillna(0)
```
""")

In [ ]:
# Ensure join keys have the same dtype to avoid merge mismatches
# Cast stop_id/trip_id/route_id to string on both sides of joins
times_gtfs_clean['trip_id'] = times_gtfs_clean['trip_id'].astype(str)
trips_gtfs_clean['trip_id'] = trips_gtfs_clean['trip_id'].astype(str)
times_gtfs_clean['stop_id'] = times_gtfs_clean['stop_id'].astype(str)
merged_stops_clean['stop_id'] = merged_stops_clean['stop_id'].astype(str)
trips_gtfs_clean['route_id'] = trips_gtfs_clean['route_id'].astype(str)

route_stops = (
    times_gtfs_clean
    .merge(trips_gtfs_clean[['trip_id', 'route_id']], on='trip_id')
    .merge(merged_stops_clean[['stop_id', 'region']], left_on='stop_id', right_on='stop_id')
    .groupby(['route_id', 'region'])
    .size()
    .reset_index(name='stop_count')
)

route_region_pivot = route_stops.pivot(
    index='route_id', 
    columns='region', 
    values='stop_count'
).fillna(0)

mr.Markdown("**Routes by Region (Pivoted):**")
route_region_pivot.head(10)

  ## Profiling: Outliers & Cardinalities

In [ ]:
mr.Markdown("""
**Outlier Detection:** Identify stops unusually far from city centre

**Cardinality Analysis:** Count unique values in key dimensions
""")

In [ ]:
print("=== OUTLIERS ===")
print("Stops beyond 15km from city centre:")
outliers = merged_stops_clean[merged_stops_clean['distance_from_centre_km'] > 15]
print(f"Found {len(outliers)} outlier stops")
if len(outliers) > 0:
    print(outliers[['stop_name', 'distance_from_centre_km']].head())

print("\n=== CARDINALITIES ===")
print(f"Unique stops: {merged_stops_clean['stop_id'].nunique()}")
print(f"Unique routes: {routes_gtfs_clean['route_id'].nunique()}")
print(f"Unique regions: {merged_stops_clean['region'].nunique()}")
print(f"Unique trips: {trips_gtfs_clean['trip_id'].nunique()}")

  ## Before/After Evidence

In [ ]:
mr.Markdown("""
### Quantitative comparison of data quality improvements:
""")

In [ ]:
print("=" * 60)
print("BEFORE (Raw Data)")
print("=" * 60)
print(f"Stop rows: {len(df_stops)}")
print(f"Route rows: {len(df_routes)}")
print(f"Missing ATSTREET: {df_stops['attributes.ATSTREET'].isnull().sum()}")
print(f"Coordinate type: {df_stops['attributes.LAT'].dtype} (string)")
print(f"Stop ID type: {df_stops['attributes.STOP_ID'].dtype} (string)")
print(f"Features: 8 columns")

print("\n" + "=" * 60)
print("AFTER (Cleaned & Transformed)")
print("=" * 60)
print(f"Stop rows: {len(merged_stops_clean)} (+{len(missing_stops)} from GTFS)")
print(f"Route rows: {len(clean_routes)}")
print(f"Missing ATSTREET: {merged_stops_clean['at_street'].isnull().sum()}")
print(f"Coordinate type: {merged_stops_clean['lat'].dtype} (numeric-ready)")
print(f"Stop ID type: {merged_stops_clean['stop_id'].dtype} (int32)")
print(f"Features: {len(merged_stops_clean.columns)} columns")
print(f"New derived features: region, route_distance_km, distance_from_centre_km")
print(f"Parsed time features: arrival_hour, arrival_minute, departure_hour, departure_minute")

  ## Visualization: Interactive Map

In [ ]:
mr.Markdown("""
**Interactive map** showing all bus stops with hover information:
""")

In [ ]:
#Adapted from the Plotly documentation https://plotly.com/python/tile-scatter-maps/#multiple-markers
stop_fig = go.Figure(go.Scattermap(
    lat=clean_stops['lat'],
    lon=clean_stops['lon'],
    mode='markers',
    marker=go.scattermap.Marker(size=9, color='blue'),
    text=clean_stops['stop_name'],
    hovertemplate='<b>%{text}</b><extra></extra>'
))

stop_fig.update_layout(
    title="Regina Transit Stops",
    autosize=True,
    hovermode='closest',
    map=dict(
        bearing=0,
        centre=dict(lat=50.447992743219615, lon=-104.61228441057489),
        pitch=0,
        zoom=11
    ),
    height=600
)

stop_fig.show()

  ## Visualization: Route Overlay

In [ ]:
mr.Markdown("""
**Adding route geometry** with coordinate transformation from UTM to lat/lon:

```python
# Transform UTM coordinates to lat/lon
transformer = Transformer.from_crs("EPSG:26913", "EPSG:4326", always_xy=True)

for coordinate in path:
    lon, lat = transformer.transform(coordinate[0], coordinate[1])
```
""")

In [ ]:
transformer = Transformer.from_crs("EPSG:26913", "EPSG:4326", always_xy=True)

route_idx = 0
route_name = clean_routes['route_name'].iloc[route_idx]
route_geometry = clean_routes['geometry_paths'].iloc[route_idx]
route_colour = clean_routes['route_color'].iloc[route_idx]

# Transform coordinates
all_lons = []
all_lats = []

for path in route_geometry:
    for coordinate in path:
        lon, lat = transformer.transform(coordinate[0], coordinate[1])
        all_lons.append(lon)
        all_lats.append(lat)
    all_lons.append(None)
    all_lats.append(None)

# Add route to map
stop_fig.add_trace(go.Scattermap(
    lon=all_lons,
    lat=all_lats,
    mode='lines',
    line=dict(width=3, color=route_colour),
    name=route_name,
    hovertemplate=f'<b>{route_name}</b><extra></extra>'
))

stop_fig.show()

  ## Summary of Transformations

In [ ]:
mr.Markdown("""
### Completed Transformations (8 operations across 5 categories):

**1. Type Fixes & Parsing**
- Converted stop_id from string to int32
- Parsed arrival/departure times to datetime
- Derived hour and minute features

**2. Text Cleanup**
- Stripped whitespace from all text columns
- Converted to uppercase for consistency
- Fixed malformed addresses

**3. Missing Data Handling**
- Imputed missing ATSTREET values
- Generated street names from stop names for GTFS stops

**4. Join/Merge**
- Merged geographic stops with GTFS schedule data
- Joined stop times → trips → routes → stops

**5. Feature Derivation**
- Created regional classifications (NE, NW, SE, SW)
- Calculated distance from city centre
- Calculate total distance of each route

**6. Aggregation**
- Summarized stops by region with statistics

**7. Reshape**
- Pivoted route-region stop counts to wide format

**8. Coordinate Transformation**
- Converted UTM to lat/lon for visualization
""")

  ## Reproducibility

In [ ]:
mr.Markdown("""
### How to reproduce this analysis:

**1. Install dependencies:**
```bash
pip install pandas plotly pyproj numpy jupyter
```

**2. Directory structure:**
```
project/
├── presentation.ipynb
├── transit_data.ipynb
├── raw_data/
│   ├── yqrStops.json
│   ├── yqrRoutes.json
│   └── gtfs_data/
│       ├── stops.txt
│       ├── routes.txt
│       ├── trips.txt
│       └── stop_times.txt
└── README.md
```

**3. Run notebook:**
- Command line: jupyter notebook transit_data.ipynb
- In VS Code: Execute all cells sequentially
- Or run with Mercury: `mercury run presentation.ipynb`

## Tool Versions

- Python: 3.14.0
- pandas: 2.3.3
- numpy: 2.3.5
- plotly: 6.5.0
- pyproj: 3.7.2
""")